In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
import time
#import codecs
#import sys
import csv
#import requests
from collections import OrderedDict

In [6]:
browser = webdriver.Chrome()
browser.get('http://seec.ct.gov/eCrisReporting/SearchingCommittee.aspx')

#creating lambda functions to click our page 'buttons'!
clickbyxpath = lambda x: browser.find_element_by_xpath(x).click()
clickbyid    = lambda x: browser.find_element_by_id(x).click()
clickbytext  = lambda x: browser.find_element_by_link_text(x).click()
goback       = lambda: browser.execute_script("window.history.go(-1)")


StateButton    = '//*[@id="ctl00_ContentPlaceHolder1_UcCommitteeAdvanceSearch1_ddlState"]/option[16]'
OfficeButton   = '//*[@id="ctl00_ContentPlaceHolder1_UcCommitteeAdvanceSearch1_ddlOfficeSought"]/option[68]'
SearchButton   = '//*[@id="ctl00_ContentPlaceHolder1_UcCommitteeAdvanceSearch1_cmdSearch"]'
nextPageButton = '//*[@id="ctl00_ContentPlaceHolder1_UcCommitteeAdvanceSearch1_cmdNext"]'

clickbyxpath(StateButton)
clickbyxpath(OfficeButton)
clickbyxpath(SearchButton)


time.sleep(3)

#### We have an issue with the below code, where our while loop, which is intended to scrape the committee details, repeatedly loops through the first page's committes, but not the remaining committees on pages 2-99.
### Error resolved 11/07/2017
While loop was searching first page repeatedly. I revised by making loop fetch the succeeding page (using pageButton) after the proceeding page was fully parsed

11/07/2018 Need to write resulting dictionaries to a CSV

In [ ]:
#loops through pages 1,99
for pagect in range(1,99):

    pageButton = '//*[@id="ctl00_ContentPlaceHolder1_UcCommitteeAdvanceSearch1_ddlGoToPage"]/option[%d]' % pagect

    time.sleep(3)
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    #isolates first entry of row 1 in table body
    #then iterates through each column of the row
    #then iterates through subsquent rows
    details = 2     

################################################################################################################
    for rowval in range(1,26):

        #this is our beautSoup of rows('tr' tags containing our rows), with 'td' tags being row elements
        tableElems = soup.find('table', {'class': 'DataTable_Grid'}).find_all('tr')[rowval].find_all('td')

        #define an array
        #rowval will be our first element of rowContents.
        rowContents = [rowval]      

        #define a second list for our detailed committee information
        candContents = []
        
        tableLength = len(tableElems)
        
##################################################################################################################
        #parse columns 1 through 12, one row at a time, for rows 1-25
        for col in range(1,tableLength):

            column_stuff = tableElems[col].find_all(string=True)
            
            #join and strip the strings
            contents = ''.join(column_stuff).strip()

            #append elements from our rows, into our list
            rowContents.append(contents)

################################################################################################################

        #clicks on candidate
        while details<=rowval+1:
            
            candidateButton = '//*[@id="ctl00_ContentPlaceHolder1_UcCommitteeAdvanceSearch1_gvSearchResult"]/tbody/tr[%d]' % details
            clickbyxpath(candidateButton)
            
            time.sleep(3)

            committee_history = BeautifulSoup(browser.page_source, 'html.parser')

            explore = committee_history.find_all('tbody')[4].find_all('tr')

            cand_name = explore[7].find('tbody')('tr')[7]('td')[8].string
            timestamp = explore[7].find_all('td')[7].find('span').string
            committee_name2 = explore[7].find_all('td')[12].string
            address1 = explore[15].find('tbody')('tr')[3]('td')[1].string
            address2 = explore[15].find('tbody')('tr')[4]('td')[1].string

            foundstuff = [timestamp, cand_name, committee_name2, address1, address2]
            candContents.extend(foundstuff)

            goback()
            time.sleep(4)
            clickbyxpath(StateButton)
            clickbyxpath(OfficeButton)
            clickbyxpath(SearchButton)
            
            time.sleep(3)

            clickbyxpath(pageButton)            
            details+=1        

################################################################################################################

        firstPage = ['row',
                     'committee_name',
                     'candidate_chairperson',
                     'treasurer',
                     'dpty_treas',
                     'office_sought',
                     'committee_type',
                     'termination_date',
                     'first_reg_date',
                     'party',
                     'district',
                     'city',
                     'state'
                     ]

        secondPage = ['timestamp',
                      'candidate',
                      'committee_name2',
                      'address1',
                      'address2'
                      ]

        desired_csv1 = dict(zip(firstPage,rowContents))
        desired_csv2 = dict(zip(secondPage, candContents))  

        print(desired_csv1)
        print('###########################')
        print(desired_csv2)
        print('###########################')


clickbyxpath(nextPageButton)
time.sleep(4)

{'row': 1, 'committee_name': 'R  95', 'candidate_chairperson': 'John Carlson', 'treasurer': "Matthew D O'Brien", 'dpty_treas': "James O'Connell", 'office_sought': 'State Representative', 'committee_type': 'Candidate Committee', 'termination_date': '', 'first_reg_date': '03/14/2018', 'party': 'Republican', 'district': '95', 'city': 'New Haven', 'state': 'CT'}
###########################
{'timestamp': '10/30/2018 5:13:00 PM ( AMENDMENT )', 'candidate': 'John Carlson', 'committee_name2': 'R  95', 'address1': '291 Greenwich Ave', 'address2': 'New Haven CT 06519 '}
###########################
{'row': 2, 'committee_name': 'Kraut For Westport', 'candidate_chairperson': 'Greg Kraut', 'treasurer': 'Robert C Bass', 'dpty_treas': '', 'office_sought': 'State Representative', 'committee_type': 'Candidate Committee', 'termination_date': '', 'first_reg_date': '05/23/2018', 'party': 'Republican', 'district': '136', 'city': 'Westport', 'state': 'CT'}
###########################
{'timestamp': '10/29/201

## Potential method for writing our dictionaries to a CSV

In [48]:
def export_dict_list_to_csv(data, filename):
    with open('statereps.csv','w') as f1, open('reps_all.csv', 'w') as f1:

        headers1 = sorted([k for k, v in desired_csv1[0].items()])
        csv_data1 = [headers1]
        
        headers = sorted([j for j, w in desired_csv2[0].items()])
        csv_data2 = [headers2]

        for d in desired_csv1:
            csv_data1.append([d[h] for h in headers1])

        writer1 = csv.writer(f1)
        writer1.writerows(csv_data1)
        
        for p in desired_csv2:
            csv_data2.append([d[h] for h in headers2])
            
        writer2 = csv.writer(f2)
        writer2.writerows(csv_data2)

In [ ]:
'''
#write our list of strings to a csv format
def writeStuff(csvWriter, foundContent):
    csvWriter.writerow(desired_csv1)
    csv
'''

with open('statereps.csv', 'wb') as f1, open('reps_all.csv', 'wb') as f2:
    dict_writer1 = csv.DictWriter(csvFile1, keys)
    dict_writer2 = csv.DictWriter(csvFile2, keys)

    dict_writer1.writeheader()
    dict_writer1.writerows(keys1)
    for d in r:
        dict_writer1.writerow(d)
        

    dict_writer2.writeheader()
    dict_writer.writerows(keys2)
    for c in q:
        dict_writer1.writerow

In [4]:

    pd
    pd.DataFrame(desired_csv1.items())
    pd.DataFrame(desired_csv2.items())

    pd.DataFrame(d.items(), columns=)

    '''
    keys1 = desired_csv1.keys()
    keys2 = desired_csv2.keys()


    with open('statereps.csv', 'w') as csvFile1, open('reps_all.csv', 'w') as csvFile2:
        dict_writer1 = csv.DictWriter(csvFile1, keys1)
        dict_writer2 = csv.DictWriter(csvFile2, keys2)

        dict_writer1.writeheader()
        dict_writer1.writerow(desired_csv1)

        dict_writer2.writeheader()
        dict_writer2.writerow(desired_csv2)
    '''

IndentationError: unexpected indent (<ipython-input-4-1b171c549b93>, line 2)